In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats
import cv2
import matplotlib.image as image
fig_DIR = "../figs/team_leaders/"
box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"
img_DIR_P = "../data/images/players/"


In [ ]:
dfa = []
for year in range(2003,2024):
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season = get_ss(year)
    )
    df1 = stats.get_data_frames()[0]
    df1 = df1.sort_values("W_PCT",ascending=False).reset_index(drop=True)
    df1["OPTS"] = df1["OFF_RATING"]*df1["POSS"]
    df1["DPTS"] = df1["DEF_RATING"]*df1["POSS"]
    lavg_Rtg = round(df1["OPTS"].sum()/df1["POSS"].sum(),2)
    df1["rOFF_RATING"] = df1["OFF_RATING"] - lavg_Rtg
    df1["rDEF_RATING"] = df1["DEF_RATING"] - lavg_Rtg
    df1["Season"] = year + 1
    dfa.append(df1)
df2 = pd.concat(dfa)

In [ ]:
df2["TEAM_NAME"] = df2["TEAM_NAME"].str.replace("Los Angeles Clippers","LA Clippers")
df2["TEAM_NAME"] = df2["TEAM_NAME"].str.replace("New Orleans/Oklahoma City Hornets","New Orleans Hornets")
df2["url"]= img_DIR_T + df2["TEAM_NAME"] + ".png"
# df3 = df2.query("OFF_RATING_RANK == 1")
df3 = df2.nsmallest(20,"rOFF_RATING")

In [ ]:
yvar = "rOFF_RATING"
p = (
    ggplot(df3,aes(x="Season",y=yvar,image="url"))
    # + geom_point(size=0.1)
    # + geom_line(size=1)
    + geom_image(size=0.07)
    # + geom_smooth(se=False,color="red",size=2)
    # + geom_image(aes(x="Season",y=yvar,image="url"),size=0.06)
    + labs(
        title=f"NBA 20 Worst Relative Offensive Ratings",
        subtitle="Last 20 Seasons",
        # subtitle="rOff Rtg = Off Rtg - League Avg Off Rtg",
        caption="@sradjoker | source:nba.com/stats",
        x="Season",
        y="rOff Rating"
    )
    # + scale_y_continuous(breaks=range(6,14))
    + scale_x_continuous(breaks=range(2004,2025,4))
    + coord_cartesian(xlim=[2004,2024])
    + theme_idv
    + theme(
        plot_title=element_text(size=14),
        # axis_title_y=element_blank(),
        figure_size=(5,4),
        # plot_margin=0.01,
    )
)
p